### Для входа нужны
- attend.csv
- groups.csv
- uniq_streets_coords.csv
- _users_features.csv

### Выход
- _attend_features.csv

In [1]:
import numpy as np
import pandas as pd

### Читаем данные

In [2]:
df_attend = pd.read_csv('attend.csv')
if 'Unnamed: 0' in df_attend.columns:
    df_attend.drop(columns=['Unnamed: 0'], inplace=True)
display(df_attend)

,уникальный номер занятия,уникальный номер группы,уникальный номер участника,направление 2,направление 3,онлайн/офлайн,дата занятия,время начала занятия,время окончания занятия,weekend
0,401346550,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00,False
1,401346550,801346550,101385462,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00,False
2,401346550,801346550,101421897,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00,False
3,401346550,801346550,101354499,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00,False
4,401346550,801346550,101421312,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00,False
...,...,...,...,...,...,...,...,...,...,...
5901269,402103132,801371145,101421020,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Да,2023-01-31,12:30:00,14:30:00,False
5901270,402103132,801371145,101359314,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Да,2023-01-31,12:30:00,14:30:00,False
5901271,402103132,801371145,101357904,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Да,2023-01-31,12:30:00,14:30:00,False
5901272,402103132,801371145,101383123,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Да,2023-01-31,12:30:00,14:30:00,False


### Группируем отдельные занятия посещения человеком, формируем новый столбец с количеством посещений

In [3]:
tmp = df_attend.groupby(['уникальный номер группы','уникальный номер участника'], as_index=False).size()
df_attend = tmp.merge(df_attend.drop_duplicates(subset=['уникальный номер группы', 'уникальный номер участника'], keep='last'), left_on=['уникальный номер группы','уникальный номер участника'], right_on=['уникальный номер группы','уникальный номер участника'], how='left')
df_attend = df_attend.drop(columns=['дата занятия','уникальный номер занятия'])
display(df_attend)

,уникальный номер группы,уникальный номер участника,size,направление 2,направление 3,онлайн/офлайн,время начала занятия,время окончания занятия,weekend
0,801346549,101370217,8,Здорово жить,Здорово жить,Нет,09:00:00,10:00:00,False
1,801346549,101375025,21,Здорово жить,Здорово жить,Нет,09:00:00,10:00:00,False
2,801346549,101392971,1,Здорово жить,Здорово жить,Нет,09:00:00,10:00:00,False
3,801346550,101352023,64,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,09:00:00,10:00:00,False
4,801346550,101354499,64,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,09:00:00,10:00:00,False
...,...,...,...,...,...,...,...,...,...
600385,801373868,101374314,2,Художественно-прикладное творчество,"Керамика (глина, тестопластика)",Нет,13:00:00,14:00:00,False
600386,801373869,101372152,1,Рисование,Различные техники рисования,Нет,15:00:00,17:00:00,False
600387,801373869,101436322,1,Рисование,Различные техники рисования,Нет,15:00:00,17:00:00,False
600388,801373870,101439415,3,Домоводство,Вторая жизнь вещей,Нет,11:00:00,13:00:00,False


### Вычисляем длительность занятия

In [4]:
df_attend['lesson_length'] = (df_attend['время окончания занятия'].astype('datetime64[ns]') - df_attend['время начала занятия'].astype('datetime64[ns]'))
df_attend['lesson_length'] = df_attend['lesson_length'] / pd.Timedelta('1 hour')

/tmp/ipykernel_12098/3308365057.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_attend['lesson_length'] = (df_attend['время окончания занятия'].astype('datetime64[ns]') - df_attend['время начала занятия'].astype('datetime64[ns]'))
/tmp/ipykernel_12098/3308365057.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_attend['lesson_length'] = (df_attend['время окончания занятия'].astype('datetime64[ns]') - df_attend['время начала занятия'].astype('datetime64[ns]'))


### Добавляем новое поле - часть дня занятия (утро, день, вечер)

In [5]:
b = [9,13,17,23]
l = ['mornig','aftennoon','evening']
df_attend['session'] = pd.cut(pd.to_datetime(df_attend['время начала занятия']).dt.hour, bins=b, labels=l, include_lowest=True)

/tmp/ipykernel_12098/1091096973.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_attend['session'] = pd.cut(pd.to_datetime(df_attend['время начала занятия']).dt.hour, bins=b, labels=l, include_lowest=True)


### Удаляем ненужные столбцы

In [6]:
df_attend = df_attend.drop(columns=['время начала занятия','время окончания занятия', 'направление 2'])

### Логарифмируем количество посещенией занятия юзером (уменьшаем разброс между большим и малым количеством занятий на разных курсах)

In [7]:
df_attend['size'] = np.log(df_attend['size'])

### Переименовываем стоблцы на английский для удобства

In [ ]:
df_attend = df_attend.rename(columns={
    'size':'log_visits',
    "уникальный номер группы": "group_id",
    "уникальный номер участника": "user_id",
    'онлайн/офлайн':'is_online'})

In [ ]:
df_attend.replace(('Да', 'Нет'), ('online', 'offline'), inplace=True)

## Обогащаем данными из таблицы с группами

In [ ]:
df_groups = pd.read_csv('groups.csv')
df_groups = df_groups[['уникальный номер','направление 1','направление 2','адрес площадки']]
display(df_groups)

### Вычисляем координаты занятия на основе адреса

In [ ]:
# Выделяем улицу
def get_street(address):
    try:
        start = address.lower().find('москва')
        address = address[start+7:]
        address = address[:address.find(',')]
    except:
        address = 'москва'
    address = address.replace('улица','').strip().lower()
    return address

df_groups['group_address'] = df_groups['адрес площадки'].apply(get_street)



### Читаем таблицу улиц-координат

In [ ]:
mid = '(55.7498, 37.5371)' # Башня Федерация

# Получаем координаты
df_uniq_streets_coords = pd.read_csv('uniq_streets_coords.csv')
df_uniq_streets_coords = df_uniq_streets_coords[['для координат','Coords']]
df_uniq_streets_coords['для координат']= df_uniq_streets_coords['для координат'].str.lower()
df_uniq_streets_coords['Coords'].fillna(mid, inplace=True)

streets_dict = dict(zip(df_uniq_streets_coords['для координат'], df_uniq_streets_coords['Coords']))

In [ ]:
display(df_groups)

### Добавляем координаты занятия (на основе адреса улицы)

In [ ]:
def match_street_coords(street):  
    res = []
    for i in streets_dict.keys():
        if(i.find(street) != -1):
            res.append(streets_dict[i])    
    if len(res) > 0:
        return res[0]
    else:
        return '(55.7498, 37.5371)'
split_data = df_groups['group_address'].apply(match_street_coords).str.strip(')').str.strip('(').str.split(',')
df_groups['lat'] = split_data.apply(lambda x: x[0]).astype(float)
df_groups['long'] = split_data.apply(lambda x: x[1]).astype(float)
df_groups = df_groups.drop(columns=['адрес площадки'])
display(df_groups)

### Добавляем в attend столбцы направление 1, направление 2

In [ ]:
df_attend = df_attend.merge(
    df_groups,
    left_on=['group_id'], 
    right_on=['уникальный номер'],
    how='left')
df_attend = df_attend.drop(columns=['уникальный номер'])

In [ ]:
display(df_attend)

## Обогащаем данными из таблицы с пользователями

In [ ]:
df_users = pd.read_csv('_users_features.csv')
df_users.drop(columns=['Unnamed: 0'],inplace=True)
display(df_users)

In [ ]:
df_attend = df_attend.merge(
    df_users,
    left_on=['user_id'], 
    right_on=['user_id'],
    how='left')


In [ ]:
display(df_attend)

### Сохраняем обогащенную таблицу

In [ ]:
df_attend.to_csv('_attend_features.csv')